In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
import torch
import matplotlib.pyplot as plt
from rowan_ds_tools.EDA import nlp

In [2]:
df = pd.read_csv('..\Data\goodreads_train.csv')

In [65]:
df['rating']

0         5
1         3
2         3
3         0
4         4
         ..
899995    3
899996    3
899997    4
899998    4
899999    3
Name: rating, Length: 900000, dtype: int64

In [3]:
tokenizer = Tokenizer(num_words=10000, oov_token='<UNK>', char_level=False)
tokenizer.fit_on_texts(df['review_text'])
model_input_dim = tokenizer.num_words

# Tokenize and Pad sequences
sequences = tokenizer.texts_to_sequences(df['review_text'])
sequences = pad_sequences(sequences,maxlen=1500, padding='post')

X = torch.tensor(sequences)

In [59]:


class RatingModel(torch.nn.Module):

    def __init__(self, model_input_dim):

        super(RatingModel,self).__init__()
        self.layer1 = torch.nn.Embedding(model_input_dim, 150) # -> (*,H)
        self.layer2 = torch.nn.GRU(150,100,num_layers=2,bidirectional=True) # -> (D*num_layers,N,H) D-2 if bidrectional, N-batch_size, H-hidden_size 
        self.layer3 = torch.nn.Linear(2*100,6)

    
    def forward(self,x):
        
        out = self.layer1(x)
        out, hidden = self.layer2(out)
        out = out[-1,:]
        out = self.layer3(out)
        return out


In [60]:
model = RatingModel(model_input_dim)

In [61]:
a = model.forward(X[0])

In [63]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 1,833,606 trainable parameters


In [103]:
df['review_text'].to_numpy()

(900000,)